In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import sklearn
import scipy
from scipy import stats
import string
import re # helps you filter urls
from IPython.display import display, Latex, Markdown
from sklearn.model_selection import train_test_split
from sklearn import metrics

# Classifying tweets [100 points]

In this problem, you will be analyzing Twitter data extracted using [this](https://dev.twitter.com/overview/api) api. The data contains tweets posted by the following six Twitter accounts: `realDonaldTrump, mike_pence, GOP, HillaryClinton, timkaine, TheDemocrats`

For every tweet, there are two pieces of information:
- `screen_name`: the Twitter handle of the user tweeting and
- `text`: the content of the tweet.

The tweets have been divided into two parts - train and test available to you in CSV files. For train, both the `screen_name` and `text` attributes were provided but for test, `screen_name` is hidden.

The overarching goal of the problem is to "predict" the political inclination (Republican/Democratic) of the Twitter user from one of his/her tweets. The ground truth (i.e., true class labels) is determined from the `screen_name` of the tweet as follows
- `realDonaldTrump, mike_pence, GOP` are Republicans
- `HillaryClinton, timkaine, TheDemocrats` are Democrats

Thus, this is a binary classification problem.

The problem proceeds in three stages:
- **Text processing (25%)**: We will clean up the raw tweet text using the various functions offered by the [nltk](http://www.nltk.org/genindex.html) package.
- **Feature construction (25%)**: In this part, we will construct bag-of-words feature vectors and training labels from the processed text of tweets and the `screen_name` columns respectively.
- **Classification (50%)**: Using the features derived, we will use [sklearn](http://scikit-learn.org/stable/modules/classes.html) package to learn a model which classifies the tweets as desired.

You will use two new python packages in this problem: `nltk` and `sklearn`, both of which should be available with anaconda. However, NLTK comes with many corpora, toy grammars, trained models, etc, which have to be downloaded manually. This assignment requires NLTK's stopwords list, POS tagger, and WordNetLemmatizer. Install them using:

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
# Verify that the following commands work for you, before moving on.

lemmatizer=nltk.stem.wordnet.WordNetLemmatizer()
stopwords=nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\saahi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\saahi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\saahi\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\saahi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Let's begin!

## A. Text Processing [25 points]

You first task to fill in the following function which processes and tokenizes raw text. The generated list of tokens should meet the following specifications:
1. The tokens must all be in lower case.
2. The tokens should appear in the same order as in the raw text.
3. The tokens must be in their lemmatized form. If a word cannot be lemmatized (i.e, you get an exception), simply catch it and ignore it. These words will not appear in the token list.
4. The tokens must not contain any punctuations. Punctuations should be handled as follows: (a) Apostrophe of the form `'s` must be ignored. e.g., `She's` becomes `she`. (b) Other apostrophes should be omitted. e.g, `don't` becomes `dont`. (c) Words must be broken at the hyphen and other punctuations.
5. The tokens must not contain any part of a url.

Part of your work is to figure out a logical order to carry out the above operations. You may find `string.punctuation` useful, to get hold of all punctuation symbols. Look for [regular expressions](https://docs.python.org/3/library/re.html) capturing urls in the text. Your tokens must be of type `str`. Use `nltk.word_tokenize()` for tokenization once you have handled punctuation in the manner specified above.

You would want to take a look at the `lemmatize()` function [here](https://www.nltk.org/_modules/nltk/stem/wordnet.html).
In order for `lemmatize()` to give you the root form for any word, you have to provide the context in which you want to lemmatize through the `pos` parameter: `lemmatizer.lemmatize(word, pos=SOMEVALUE)`. The context should be the part of speech (POS) for that word. The good news is you do not have to manually write out the lexical categories for each word because [nltk.pos_tag()](https://www.nltk.org/book/ch05.html) will do this for you. Now you just need to use the results from `pos_tag()` for the `pos` parameter.
However, you can notice the POS tag returned from `pos_tag()` is in different format than the expected pos by `lemmatizer`.
> pos
(Syntactic category): n for noun files, v for verb files, a for adjective files, r for adverb files.

You need to map these pos appropriately. `nltk.help.upenn_tagset()` provides description of each tag returned by `pos_tag()`.



## Q1 (15 points):

In [3]:
# Convert part of speech tag from nltk.pos_tag to word net compatible format
# Simple mapping based on first letter of return tag to make grading consistent
# Everything else will be considered noun 'n'
posMapping = {
# "First_Letter by nltk.pos_tag":"POS_for_lemmatizer"
    "N":'n',
    "V":'v',
    "J":'a',
    "R":'r'
}
# 14% credits
def process(text, lemmatizer=nltk.stem.wordnet.WordNetLemmatizer()):
    """ Normalizes case and handles punctuation
    Inputs:
        text: str: raw text
        lemmatizer: an instance of a class implementing the lemmatize() method
                    (the default argument is of type nltk.stem.wordnet.WordNetLemmatizer)
    Outputs:
        list(str): tokenized text
    """

    
    url_pattern = r'https?://\S+|www\.\S+'
    text = re.sub(url_pattern, "", text)

    if text.find("'s") > 0:
        text = text.replace("'s", "")

    if text.find("-") > 0:
        text = text.replace("-", " ")

    if text.find("'") > 0:
        text = text.replace("'", "")

    for i in text:
        if i in string.punctuation:
            text = text.replace(i, " ")

    tokenized_text = nltk.word_tokenize(text)

    pos_tag_list = nltk.pos_tag(tokenized_text)
    
    newList = []
    for pair in pos_tag_list:
        string1, string2 = pair
        if "N" == string2[0]:
            newList.append((string1, 'n'))
        elif "V" == string2[0]:
            newList.append((string1, 'v'))
        elif "J" == string2[0]:
            newList.append((string1, 'a'))
        elif "R" == string2[0]:
            newList.append((string1, 'r'))
        else:
            newList.append((string1, 'n'))

    lemmatized_tokens = []
    for pair in newList:
        string1, string2 = pair
        lemmatized_tokens.append(lemmatizer.lemmatize(string1.lower(), pos=string2))
    
    # print("my answer:", lemmatized_tokens)
    return lemmatized_tokens
    

You can test the above function as follows. Try to make your test strings as exhaustive as possible. Some checks are:

In [4]:
# 1 point credit
print(process("I'm doing well! How about you?"))
# print("Answer:", ['im', 'do', 'well', 'how', 'about', 'you'])

print(process("Education is the ability to listen to almost anything without losing your temper or your self-confidence."))
# print("Answer:", ['education', 'be', 'the', 'ability', 'to', 'listen', 'to', 'almost', 'anything', 'without', 'lose', 'your', 'temper', 'or', 'your', 'self', 'confidence'])

print(process("been had done languages cities mice"))
# print("Answer:", ['be', 'have', 'do', 'language', 'city', 'mice'])

print(process("It's hilarious. Check it out http://t.co/dummyurl"))
# print("Answer:", ['it', 'hilarious', 'check', 'it', 'out'])

print(process("See it Sunday morning at 8:30a on RTV6 and our RTV6 app. http:…"))
# print("Answer:", ['see', 'it', 'sunday', 'morning', 'at', '8', '30a', 'on', 'rtv6', 'and', 'our', 'rtv6', 'app', 'http', '…'])
# Here '…' is a special unicode character not in string.punctuation and it is still present in processed text

['im', 'do', 'well', 'how', 'about', 'you']
['education', 'be', 'the', 'ability', 'to', 'listen', 'to', 'almost', 'anything', 'without', 'lose', 'your', 'temper', 'or', 'your', 'self', 'confidence']
['be', 'have', 'do', 'language', 'city', 'mice']
['it', 'hilarious', 'check', 'it', 'out']
['see', 'it', 'sunday', 'morning', 'at', '8', '30a', 'on', 'rtv6', 'and', 'our', 'rtv6', 'app', 'http', '…']




## Q2 (10 points):

You will now use the `process()` function we implemented to convert the pandas dataframe we just loaded from tweets_train.csv file. Your function should be able to handle any data frame which contains a column called `text`. The data frame you return should replace every string in `text` with the result of `process()` and retain all other columns as such. Do not change the order of rows/columns. Before writing `process_all()`, load the data into a DataFrame and look at its format:

In [5]:
tweets = pd.read_csv("tweets_train.csv", na_filter=False)
display(tweets.head())

,screen_name,text
0,GOP,RT @GOPconvention: #Oregon votes today. That m...
1,TheDemocrats,RT @DWStweets: The choice for 2016 is clear: W...
2,HillaryClinton,Trump's calling for trillion dollar tax cuts f...
3,HillaryClinton,.@TimKaine's guiding principle: the belief tha...
4,timkaine,Glad the Senate could pass a #THUD / MilCon / ...


In [6]:
# 9 points credits
def process_all(df, lemmatizer=nltk.stem.wordnet.WordNetLemmatizer()):
    """ process all text in the dataframe using process() function.
    Inputs
        df: pd.DataFrame: dataframe containing a column 'text' loaded from the CSV file
        lemmatizer: an instance of a class implementing the lemmatize() method
                    (the default argument is of type nltk.stem.wordnet.WordNetLemmatizer)
    Outputs
        pd.DataFrame: dataframe in which the values of text column have been changed from str to list(str),
                        the output from process() function. Other columns are unaffected.
    """

    df['text'] = df['text'].apply(process)

    return df
    
    

In [7]:
# test your code
# 1 point credit
processed_tweets = process_all(tweets)
print(processed_tweets.head())

# data = {'text': ['Hello all I\'m living', 'This world hasn\'t been the same', 'Amber is this python or not?']}
# df = pd.DataFrame(data)
# print(df.head())
# print(process_all(df))


#       screen_name                                               text
# 0             GOP  [rt, gopconvention, oregon, vote, today, that,...
# 1    TheDemocrats  [rt, dwstweets, the, choice, for, 2016, be, cl...
# 2  HillaryClinton  [trump, call, for, trillion, dollar, tax, cut,...
# 3  HillaryClinton  [timkaine, guide, principle, the, belief, that...
# 4        timkaine  [glad, the, senate, could, pass, a, thud, milc...

      screen_name                                               text
0             GOP  [rt, gopconvention, oregon, vote, today, that,...
1    TheDemocrats  [rt, dwstweets, the, choice, for, 2016, be, cl...
2  HillaryClinton  [trump, call, for, trillion, dollar, tax, cut,...
3  HillaryClinton  [timkaine, guide, principle, the, belief, that...
4        timkaine  [glad, the, senate, could, pass, a, thud, milc...


In [8]:
processed_tweets

,screen_name,text
0,GOP,"[rt, gopconvention, oregon, vote, today, that,..."
1,TheDemocrats,"[rt, dwstweets, the, choice, for, 2016, be, cl..."
2,HillaryClinton,"[trump, call, for, trillion, dollar, tax, cut,..."
3,HillaryClinton,"[timkaine, guide, principle, the, belief, that..."
4,timkaine,"[glad, the, senate, could, pass, a, thud, milc..."
...,...,...
17293,mike_pence,"[rt, govpencein, cast, a, line, w, firstladyin..."
17294,mike_pence,"[rt, indiana, edc, indiana, be, at, record, em..."
17295,mike_pence,"[rt, indiana, edc, indy, will, have, a, differ..."
17296,HillaryClinton,"[tune, in, now, to, watch, joebiden, hit, the,..."


## B. Feature Construction [25 points]

The next step is to derive feature vectors from the tokenized tweets. In this section, you will be constructing a bag-of-words TF-IDF feature vector. But before that, as you may have guessed, the number of possible words is prohibitively large and not all of them may be useful for our classification task. We need to determine which words to retain, and which to omit. A common heuristic is to construct a frequency distribution of words in the corpus and prune out the head and tail of the distribution. The intuition of the above operation is as follows. Very common words (i.e. stopwords) add almost no information regarding similarity of two pieces of text. Similarly with very rare words. NLTK has a list of in-built stop words which is a good substitute for head of the distribution. We will consider a word rare if it occurs only in a single document (row) in whole of `tweets_train.csv`.

## Q3 (15 points):

Construct a sparse matrix of features for each tweet with the help of `sklearn.feature_extraction.text.TfidfVectorizer` (documentation [here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)). You need to pass a parameter `min_df=2` to filter out the words occuring only in one document in the whole training set. Remember to ignore the stop words as well. You must leave other optional parameters (e.g., `vocab`, `norm`, etc) at their default values. But you may need to use parameters like `lowercase` and `tokenizer` to handle `processed_tweets` that is a `list` of tokens (not raw text).

In [9]:
# 14 points credits
def create_features(processed_tweets, stop_words):
    """ creates the feature matrix using the processed tweet text
    Inputs:
        processed_tweets: pd.DataFrame: processed tweets read from train/test csv file, containing the column 'text'
        stop_words: list(str): stop_words by nltk stopwords (after processing)
    Outputs:
        sklearn.feature_extraction.text.TfidfVectorizer: the TfidfVectorizer object used
            we need this to tranform test tweets in the same way as train tweets
        scipy.sparse.csr.csr_matrix: sparse bag-of-words TF-IDF feature matrix
    """
    def tokenize(text):
        return text

    stop_word_list = list(stop_words)
    sorted_stop_word_list = sorted(stop_word_list)

    # Constructing vectorizer
    vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(lowercase=False, tokenizer=tokenize, stop_words=sorted_stop_word_list, min_df=2)
    tfidf_matrix = vectorizer.fit_transform(processed_tweets['text'])

    # Constructing the CSR
    tfidf_matrix_csr = scipy.sparse.csr.csr_matrix(tfidf_matrix)

    # return vectorizer and CSR
    return vectorizer, tfidf_matrix_csr


In [10]:
# execute this code
# 1 point credit
# It is recommended to process stopwords according to our data cleaning rules
processed_stopwords = set(np.concatenate([process(word) for word in stopwords]))
(tfidf, X) = create_features(processed_tweets, processed_stopwords)
# Ignore warning
tfidf, X
# Output (should be similar):
# (TfidfVectorizer(lowercase=False, min_df=2,
#                  stop_words={'a', 'about', 'above', 'after', 'again', 'against',
#                              'ain', 'all', 'an', 'and', 'any', 'aren', 'arent',
#                              'at', 'be', 'because', 'before', 'below', 'between',
#                              'both', 'but', 'by', 'can', 'couldn', 'couldnt',
#                              'd', 'didn', 'didnt', 'do', 'doesn', ...},
#                  tokenizer=<function create_features.<locals>.<lambda> at 0x7fd4002a6700>),
#  <17298x8114 sparse matrix of type '<class 'numpy.float64'>'
#  	with 170355 stored elements in Compressed Sparse Row format>)

c:\Users\saahi\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\saahi\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'e', 'f', 'g', 'h', 'j', 'l', 'n', 'p', 'r', 'u', 'v', 'w'] not in stop_words.
  warnings.warn(
C:\Users\saahi\AppData\Local\Temp\ipykernel_27368\1690778641.py:23: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  tfidf_matrix_csr = scipy.sparse.csr.csr_matrix(tfidf_matrix)


(TfidfVectorizer(lowercase=False, min_df=2,
                 stop_words=['a', 'about', 'above', 'after', 'again', 'against',
                             'ain', 'all', 'an', 'and', 'any', 'aren', 'arent',
                             'at', 'be', 'because', 'before', 'below', 'between',
                             'both', 'but', 'by', 'can', 'couldn', 'couldnt',
                             'd', 'didn', 'didnt', 'do', 'doesn', ...],
                 tokenizer=<function create_features.<locals>.tokenize at 0x00000277F3D9EFC0>),
 <17298x8167 sparse matrix of type '<class 'numpy.float64'>'
 	with 169355 stored elements in Compressed Sparse Row format>)



## Q4 (10%):

Also for each tweet, assign a class label (0 or 1) using its `screen_name`. Use 0 for realDonaldTrump, mike_pence, GOP and 1 for the rest.

In [11]:
display(processed_tweets)

,screen_name,text
0,GOP,"[rt, gopconvention, oregon, vote, today, that,..."
1,TheDemocrats,"[rt, dwstweets, the, choice, for, 2016, be, cl..."
2,HillaryClinton,"[trump, call, for, trillion, dollar, tax, cut,..."
3,HillaryClinton,"[timkaine, guide, principle, the, belief, that..."
4,timkaine,"[glad, the, senate, could, pass, a, thud, milc..."
...,...,...
17293,mike_pence,"[rt, govpencein, cast, a, line, w, firstladyin..."
17294,mike_pence,"[rt, indiana, edc, indiana, be, at, record, em..."
17295,mike_pence,"[rt, indiana, edc, indy, will, have, a, differ..."
17296,HillaryClinton,"[tune, in, now, to, watch, joebiden, hit, the,..."


In [12]:
# 9 point credits
def create_labels(processed_tweets):
    """ creates the class labels from screen_name
    Inputs:
        processed_tweets: pd.DataFrame: tweets read from train file, containing the column 'screen_name'
    Outputs:
        numpy.ndarray(int): dense binary numpy array of class labels
    """

    # Get a list of the screen names
    screen_names = processed_tweets['screen_name'].tolist()

    # Define class labels
    class_labels_mapping = {
        'GOP': 0,
        'realDonaldTrump': 0,
        'mike_pence': 0,
        'HillaryClinton': 1,
        'timkaine': 1,
        'TheDemocrats': 1
    }

    # Map screen names to class labels
    class_labels = [class_labels_mapping[screen_name] for screen_name in screen_names]

    # Convert class labels to Pandas Series
    class_labels_series = pd.Series(class_labels, name='screen_name')

    return class_labels_series.to_numpy().astype(np.int32)

In [13]:
# execute this code
# 1 point credit
y = create_labels(processed_tweets)
y
# 0        0
# 1        1
# 2        1
# 3        1
# 4        1
#         ..
# 17293    0
# 17294    0
# 17295    0
# 17296    1
# 17297    0
# Name: screen_name, Length: 17298, dtype: int32

array([0, 1, 1, ..., 0, 1, 0])

## C. Classification [50 points]

And finally, we are ready to put things together and learn a model for the classification of tweets. The classifier you will be using is [`sklearn.svm.SVC`](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC) (Support Vector Machine).

At the heart of SVMs is the concept of kernel functions, which determines how the similarity/distance between two data points in computed. `sklearn`'s SVM provides four kernel functions: `linear`, `poly`, `rbf`, `sigmoid` (details [here](http://scikit-learn.org/stable/modules/svm.html#svm-kernels)) but you can also implement your own distance function and pass it as an argument to the classifier.

Through the various functions you implement in this part, you will be able to learn a classifier, score a classifier based on how well it performs, use it for prediction tasks and compare it to a baseline.

Specifically, you will carry out the following tasks (Q5-9) in order:

1. Implement and evaluate a simple baseline classifier MajorityLabelClassifier.
2. Implement the `learn_classifier()` function assuming `kernel` is always one of {`linear`, `poly`, `rbf`, `sigmoid`}.
3. Implement the `evaluate_classifier()` function which scores a classifier based on accuracy of a given dataset.
4. Implement `best_model_selection()` to perform cross-validation by calling `learn_classifier()` and `evaluate_classifier()` for different folds and determine which of the four kernels performs the best.
5. Go back to `learn_classifier()` and fill in the best kernel.


## Q5 (10 points):

To determine whether your classifier is performing well, you need to compare it to a baseline classifier. A baseline is generally a simple or trivial classifier and your classifier should beat the baseline in terms of a performance measure such as accuracy. Implement a classifier called `MajorityLabelClassifier` that always predicts the class equal to **mode** of the labels (i.e., the most frequent label) in training data. Part of the code is done for you. Implement the `fit` and `predict` methods. Initialize your classifier appropriately.

In [14]:
# Skeleton of MajorityLabelClassifier is consistent with other sklearn classifiers
# 8 points credits
class MajorityLabelClassifier():
    """
    A classifier that predicts the mode of training labels
    """
    def __init__(self):
        """
        Initialize your parameter here
        """
        y_mode = 0

    def fit(self, X, y):
        """
        Implement fit by taking training data X and their labels y and finding the mode of y
        i.e. store your learned parameter
        """
        self.y_mode = scipy.stats.mode(y)[0]
        return self.y_mode

    def predict(self, X):
        """
        Implement to give the mode of training labels as a prediction for each data instance in X
        return labels
        """
        return [self.y_mode] * X.shape[0]

# 2 points credits
# Report the accuracy of your classifier by comparing the predicted label of each example to its true label
baselineClf = MajorityLabelClassifier()
# Use fit and predict methods to get predictions and compare it with the true labels y

# Using fit

y_fit = baselineClf.fit(X, y)

# Using predict
test_tweets = pd.read_csv("tweets_test.csv", na_filter=False)

processed_test_tweets = process_all(test_tweets)
processed_stopwords = set(np.concatenate([process(word) for word in stopwords]))
(tfidf_test, X_test) = create_features(processed_test_tweets, processed_stopwords)

y_predict = baselineClf.predict(X_test)

# print(training accuracy) should give 0.5001734304543878
score = np.mean(y)
print("training accuracy:", score)

training accuracy: 0.5001734304543878


c:\Users\saahi\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\saahi\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'e', 'f', 'g', 'h', 'j', 'l', 'n', 'p', 'r', 'u', 'v', 'w'] not in stop_words.
  warnings.warn(
C:\Users\saahi\AppData\Local\Temp\ipykernel_27368\1690778641.py:23: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  tfidf_matrix_csr = scipy.sparse.csr.csr_matrix(tfidf_matrix)



## Q6 (10 points):

Implement the `learn_classifier()` function assuming `kernel` is always one of {`linear`, `poly`, `rbf`, `sigmoid`}. Stick to default values for any other optional parameters.

Hint: Check https://scikit-learn.org/stable/modules/svm.html#svm-kernels on how to use sklearn.svm.SVC

In [15]:
# 9 points credits

def learn_classifier(X_train, y_train, kernel):
    """ learns a classifier from the input features and labels using the kernel function supplied
    Inputs:
        X_train: scipy.sparse.csr.csr_matrix: sparse matrix of features, output of create_features(), X
        y_train: numpy.ndarray(int): dense binary vector of class labels, output of create_labels(), y
        kernel: str: kernel function to be used with classifier. [linear|poly|rbf|sigmoid]
    Outputs:
        sklearn.svm.SVC: classifier learnt from data
    """

    # Create SVM classifier
    clf = sklearn.svm.SVC(kernel=kernel)

    # Train the classifier
    clf.fit(X_train, y_train)

    return clf
    

In [16]:
# execute code
# 1 point credit
classifier = learn_classifier(X, y, 'linear')



## Q7 (10 points):

Now that we know how to learn a classifier, the next step is to evaluate it, ie., characterize how good its classification performance is. This step is necessary to select the best model among a given set of models, or even tune hyperparameters for a given model.

There are two questions that should now come to your mind:
1. **What data to use?**
    - **Validation Data**: The data used to evaluate a classifier is called **validation data** (or hold-out data), and it is usually different from the data used for training. The model or hyperparameter with the best performance in the held out data is chosen. This approach is relatively fast and simple but vulnerable to biases found in validation set.
    - **Cross-validation**: This approach divides the dataset in $k$ groups (so, called k-fold cross-validation). One of group is used as test set for evaluation and other groups as training set. The model or hyperparameter with the best average performance across all k folds is chosen. For this question you will perform 4-fold cross validation to determine the best kernel. We will keep all other hyperparameters default for now. This approach provides robustness toward biasness in validation set. However, it takes more time.
    
2. **And what metric?** There are several evaluation measures available in the literature (e.g., accuracy, precision, recall, F-1,etc) and different fields have different preferences for specific metrics due to different goals. We will go with accuracy. According to wiki, **accuracy** of a classifier measures the fraction of all data points that are correctly classified by it; it is the ratio of the number of correct classifications to the total number of (correct and incorrect) classifications. `sklearn.metrics` provides a number of performance metrics.

Now, implement the following function.

In [17]:
# 9 points credits
def evaluate_classifier(classifier, X_validation, y_validation):
    """ evaluates a classifier based on a supplied validation data
    Inputs:
        classifier: sklearn.svm.classes.SVC: classifer to evaluate
        X_validation: scipy.sparse.csr.csr_matrix: sparse matrix of features
        y_validation: numpy.ndarray(int): dense binary vector of class labels
    Outputs:
        double: accuracy of classifier on the validation data
    """
    y_predict = classifier.predict(X_validation)

    return sklearn.metrics.accuracy_score(y_validation, y_predict)

In [17]:
# test your code by evaluating the accuracy on the training data
# 1 point credit
accuracy = evaluate_classifier(classifier, X, y)
print(accuracy)
# should give around 0.9545034107989363

0.9547346514047866




## Q8 (10 points):

Now it is time to decide which kernel works best by using the cross-validation technique. Write code to split the training data into 4-folds (75% training and 25% validation) by shuffling randomly. For each kernel, record the average accuracy for all folds and determine the best classifier. Since our dataset is balanced (both classes are in almost equal propertion), `sklearn.model_selection.KFold` [doc](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) can be used for cross-validation.

In [18]:
kf = sklearn.model_selection.KFold(n_splits=4, random_state=1, shuffle=True)
kf

KFold(n_splits=4, random_state=1, shuffle=True)

Then use the following code to determine which classifier is the best.

In [19]:
# 10 points credits
def best_model_selection(kf, X, y):
    """
    Select the kernel giving best results using k-fold cross-validation.
    Other parameters should be left default.
    Input:
    kf (sklearn.model_selection.KFold): kf object defined above
    X (scipy.sparse.csr.csr_matrix): training data
    y (array(int)): training labels
    Return:
    best_kernel (string)
    """
    # kernel_accuracies = {'linear': 0, 'rbf': 0, 'poly': 0, 'sigmoid': 0}
    best_kernel = ''
    best_accuracy = 0
    for kernel in ['linear', 'rbf', 'poly', 'sigmoid']:
        # Use the documentation of KFold cross-validation to split ..
        # training data and test data from create_features() and create_labels()
        # call learn_classifer() using training split of kth fold
        # evaluate on the test split of kth fold
        # record avg accuracies and determine best model (kernel)
        #return best kernel as string
        accuracy = 0
        for train_index, val_index in kf.split(X):
            X_train, X_val = X[train_index], X[val_index]
            y_train, y_val = y[train_index], y[val_index]

            clf = learn_classifier(X_train, y_train, kernel)
            accuracy += evaluate_classifier(clf, X_val, y_val)

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_kernel = kernel

    return best_kernel

#Test your code
best_kernel = best_model_selection(kf, X, y)
best_kernel

KeyboardInterrupt: 


## Q9 (10 points)

We're almost done! It's time to write a nice little wrapper function that will use our model to classify unlabeled tweets from tweets_test.csv file.

In [20]:
# 8points credits
def classify_tweets(tfidf, classifier, unlabeled_tweets):
    """ predicts class labels for raw tweet text
    Inputs:
        tfidf: sklearn.feature_extraction.text.TfidfVectorizer: the TfidfVectorizer object used on training data
        classifier: sklearn.svm.SVC: classifier learned
        unlabeled_tweets: pd.DataFrame: tweets read from tweets_test.csv
    Outputs:
        numpy.ndarray(int): dense binary vector of class labels for unlabeled tweets
    """
    
    # svm_clf = learn_classifier(X, y, 'poly')
    # accuracy = evaluate_classifier(svm_clf, X, y)
    # process_all(unlabeled_tweets)

    new_df = process_all(unlabeled_tweets)
    matrix = tfidf.transform(new_df['text']) #ncreate a matrix which contains tfidf of each word
    return classifier.predict(matrix) # predict the label
    

Did your SVM classifier perform better than the baseline (while evaluating with training data)? Explain in 1-2 sentences how you reached this conclusion.

Below, I have compared the accuracy from the baseline and my poly SVM classifier. The poly one is better as its accuracy is higher than the linear one.

In [21]:
classifier = learn_classifier(X, y, 'poly')
unlabeled_tweets = pd.read_csv("tweets_test.csv", na_filter=False)
y_pred = classify_tweets(tfidf, classifier, unlabeled_tweets)
print(y_pred)

poly_score = evaluate_classifier(classifier, X, y)

if (score < poly_score):
    print("poly is better")
else:
    print("linear is better")

[1 1 1 1 1 1 0 1 0 0 1 1 1 1 1 1 0 0 1 0 0 1 0 0 0 0 0 1 0 0 1 0 1 1 1 1 0
 0 1 1 0 0 0 1 1 1 0 0 0 1 0 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 1 1 1 0 0 1 1 1
 1 1 1 1 1 0 0 1 0 1 0 1 1 0 1 0 1 0 0 0 1 0 0 1 0 1 1 1 1 1 1 0 0 0 0 1 0
 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 0 1 1 1 1 1 0 1 0 0 0 1 1 0 0 1
 0 1 0 1 0 0 1 1 0 1 0 1 1 0 0 0 1 1 1 1 0 1 1 0 1 1 0 0 1 0 0 0 1 0 1 0 0
 0 1 0 1 1 0 1 0 1 1 1 0 0 0 1 0 0 0 0 1 0 0 1 0 1 0 1 1 1 1 0 0 0 0 1 1 0
 0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 1 0 1 0 0 0 1 1 0 1 1 0 0 1 1 1 0 1
 0 0 0 0 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 0 0 1 1 1 0 1 0 0 1 1 0 1 1 0 1 1 1
 1 0 1 1 1 1 0 1 0 1 0 1 1 0 0 1 1 1 1 0 0 1 0 1 1 0 0 0 1 1 1 0 0 1 0 1 0
 1 0 1 1 1 1 1 0 0 1 1 0 0 1 0 0 0 0 1 1 0 1 1 1 0 0 1 1 0 1 1 1 1 0 1 0 1
 0 0 1 1 1 0 1 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 1 0 0 1 0 1 0 0 1 1 0 0 1
 1 0 1 1 1 1 0 0 1 0 1 1 0 0 0 1 1 0 1 1 1 0 1 0 1 1 0 0 1 1 0 1 1 0 0 0 1
 0 0 1 0 0 0 1 1 1 1 0 0 1 1 1 1 0 1 1 0 0 0 1 1 0 1 0 0 1 0 0 0 1 0 0 1 1
 0 0 1 0 0 1 0 0 0 1 0 1 